[View in Colaboratory](https://colab.research.google.com/github/sanattaori/Recommender-System/blob/master/MovieRecommendation.ipynb)

# Movie Recommendation System using lightfm

In [1]:
!pip install lightfm

In [0]:
# imports
import numpy as np

from lightfm.datasets import fetch_movielens
from lightfm import LightFM

### Movielens dataset 
*https://github.com/lyst/lightfm/blob/master/lightfm/datasets/movielens.py#L107*


In [0]:
# dataset
data = fetch_movielens(min_rating=4.0)


In [4]:
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


In [5]:
print(data)

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>, 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>, 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
	with 1682 stored elements in Compressed Sparse Row format>, 'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object), 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)}


### Using wrap loss function
*https://github.com/lyst/lightfm/blob/master/lightfm/lightfm.py#L53*

In [7]:
# create model using Weighted Approximate-Rank Pairwise loss function

model = LightFM(loss='warp')

#train model
model.fit(data['train'], epochs=30, num_threads=2)


def sample_recommendation(model, data, user_ids):

    #number of users and movies in training data
    n_users, n_items = data['train'].shape

    #generate recommendations for each user we input
    for user_id in user_ids:

        #movies they already like
        user_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

        #movies our model predicts they will like
        scores = model.predict(user_id, np.arange(n_items))
        
        #rank them in order of most liked to least
        top_items = data['item_labels'][np.argsort(-scores)]

        #print out the results
        print("User %s" % user_id)
        print("Users knowns")

        for x in user_positives[:3]:
            print("           %s" % x)

        print("user Recommended:")

        for x in top_items[:3]:
            print("           Ranking %s" % x)
            
sample_recommendation(model, data, [3, 25, 450])

User 3
Users knowns
           Seven (Se7en) (1995)
           Contact (1997)
           Starship Troopers (1997)
user Recommended:
           Ranking Scream (1996)
           Ranking Air Force One (1997)
           Ranking Game, The (1997)
User 25
Users knowns
           Dead Man Walking (1995)
           Star Wars (1977)
           Fargo (1996)
user Recommended:
           Ranking Contact (1997)
           Ranking Fargo (1996)
           Ranking English Patient, The (1996)
User 450
Users knowns
           Contact (1997)
           George of the Jungle (1997)
           Event Horizon (1997)
user Recommended:
           Ranking Air Force One (1997)
           Ranking Conspiracy Theory (1997)
           Ranking Game, The (1997)
